In [ ]:
# =============================================================================
# Aplicar imputación
# =============================================================================
from classes.DuckDB_Helper_v02 import DuckDBHelper 
from classes.Tronaduras_File_Reader_v03 import TronadurasFileReader
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

#╭──────────────────────────────────────────────────────╮
#| Preprocesamiento de datos                            | 
#╰──────────────────────────────────────────────────────╯
# 📌 Nombre de la base de datos DuckDB
DB_NAME = "data/Tronaduras_vs_Sismicidad.db"
SCHEMA_NAME = "Raw_Data"
CONJUNTO_ESPECIFICO = "Tronaduras"

# 🔗 Instanciar y conectar
db_helper = DuckDBHelper(DB_NAME)

# Instanciar la clase de lectura de tronaduras
Tronaduras_Reader = TronadurasFileReader()

# Leer la tabla completa Raw_Data.Tronaduras usando la clase DuckDBHelper
df_sismicidad = db_helper.select_df(table="Sismicidad",  schema="Raw_Data")

# Leer la tabla completa Raw_Data.Tronaduras usando la clase DuckDBHelper
df_tronaduras = db_helper.select_df(table="Tronaduras",  schema="Raw_Data")


#╭──────────────────────────────────────────────────────────────╮
#| Completar datos faltantes o imputación a la Tabla Tronaduras | 
#╰──────────────────────────────────────────────────────────────╯
# 🔍 Leer estrategias de imputacion
strategies_df = db_helper.select_df(
    table="Variables_Description",
    columns='"Variable", "Imputacion", "Defecto"',
    where=f'Conjunto = \'{CONJUNTO_ESPECIFICO}\'',  # 🔥 Filtra por conjunto
    schema=SCHEMA_NAME
)

# Aplicar la imputación
df_tronaduras = Tronaduras_Reader.impute_df(df=df_tronaduras, strategies_df=strategies_df)


#╭──────────────────────────────────────────────────────────────────────────╮
#| Crear nuevas variables de la tabla Sismicidad:                           |
#| Energia y Momento Sismico(Mo) Acumulados                                 |
#╰──────────────────────────────────────────────────────────────────────────╯

# Ejemplo de columna de Momento Sísmico (ajusta la fórmula a tu caso)
df_sismicidad['Mo'] = 10 ** ((3/2) * (df_sismicidad['Local Magnitude'] + 6.1))
# df_sismicidad['Mo'] = ((3/2) * (df_sismicidad['Local Magnitude'] + 6.1))

Graficos = False
if Graficos:
    # Gráfico del Momento Sísmico (Mo)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_sismicidad['Date Time'], y=df_sismicidad['Mo'], mode='lines', name='Mo'))
    fig.update_layout(title='Momento Sísmico (Mo) a lo largo del tiempo',
                    xaxis_title='Fecha',
                    yaxis_title='Mo')
    fig.show()

    # Gráfico de la Energía
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_sismicidad['Date Time'], y=df_sismicidad['Energy [J]'], mode='lines', name='Energía'))
    fig.update_layout(title='Energía a lo largo del tiempo',
                    xaxis_title='Fecha',
                    yaxis_title='Energía')
    fig.show()


#╭────────────────────────────────────────────────────────────────────╮
#| Relacionar la tabla Tronaduras con la tabla Sismicidad             |
#|    Crear una tabla de relación entre ambas tablas usando:          |
#|    - "Date Time": Fecha y hora de la tronadura y evento sísmico    |
#|       posterior a la fechas de tronadura y antes de la fecha de    |
#|        la siguiente tronadura o un rango de tiempo definido.       |
#╰────────────────────────────────────────────────────────────────────╯

# Crear nuevas columnas para almacenar el N° Disparo de las Tronaduras y su Date Time en df_sismicidad
hours_after = 'automatic' # Valor 0: no se considera el rango de tiempo, Valor 'automatic': se calcula automáticamente
df_sismicidad_con_disparo = df_sismicidad.copy()
df_sismicidad_con_disparo['N° Disparo'] = None
df_sismicidad_con_disparo['Disparo - Date Time'] = None

if hours_after=='automatic':
    print("⏲ Calculo automatico del delta entre disparos")
    # Calcular el delta entre fecha_inicio y fecha_fin (en horas)
    deltas = []
    # Iterar por cada disparo en df_tronadura para asignar los rangos
    for i in range(len(df_tronaduras) - 1):
        disparo_actual = df_tronaduras.loc[i, 'N° Disparo']
        fecha_inicio = df_tronaduras.loc[i, 'Fecha']
        fecha_fin = df_tronaduras.loc[i + 1, 'Fecha']
        
        # Calcular el delta entre fecha_inicio y fecha_fin (en horas)
        delta = (fecha_fin - fecha_inicio)
        deltas.append(delta)
    print(f"⏲ Delta max: {int(max(deltas).total_seconds()/3600)} hrs. \t | Delta min: {int(min(deltas).total_seconds()/3600)} hrs. \t | Delta promedio: {int(np.mean(deltas).total_seconds()/3600)} hrs.")

    # obtener el delta_minimo en horas truncando los minutos
    hours_after = int(min(deltas).total_seconds() / 3600)
    print(f"⏲ Delta automático: {hours_after} hrs.:")

deltas_calculados = []
# Iterar por cada disparo en df_tronadura para asignar los rangos
for i in range(len(df_tronaduras) - 1): 
    disparo_actual = df_tronaduras.loc[i, 'N° Disparo']
    fecha_inicio = df_tronaduras.loc[i, 'Fecha']
    fecha_fin = fecha_inicio + pd.Timedelta(hours=hours_after) if hours_after>0 else df_tronaduras.loc[i + 1, 'Fecha']
    
    # Calcular el delta entre fecha_inicio y fecha_fin (en horas)
    delta = (fecha_fin - fecha_inicio)
    deltas_calculados.append(delta)

    # Filtrar los valores en df_sismicidad que están dentro del rango
    mask = (df_sismicidad_con_disparo['Date Time'] >= fecha_inicio) & (df_sismicidad_con_disparo['Date Time'] < fecha_fin)
    n_eventos = len(df_sismicidad_con_disparo.loc[mask])
    df_sismicidad_con_disparo.loc[mask, 'N° Disparo'] = disparo_actual
    df_sismicidad_con_disparo.loc[mask, 'Disparo - Date Time'] = fecha_inicio


    print(f"\t 🕒 {disparo_actual} -> Delta: {fecha_inicio} - {fecha_fin}: {int(delta.total_seconds()/3600)} hrs. - "+
      f" Numero de eventos: {n_eventos}")
print(f"⏲ Delta max: {int(max(deltas_calculados).total_seconds()/3600)} hrs. \t |"+
      f" Delta min: {int(min(deltas_calculados).total_seconds()/3600)}hrs. \t |"+
      f" Delta promedio: {int(np.mean(deltas_calculados).total_seconds()/3600)} hrs.")

# Mover las columnas 'N° Disparo' y 'Disparo Date Time' al inicio
columns_order = ['N° Disparo', 'Disparo - Date Time'] + [col for col in df_sismicidad_con_disparo.columns if col not in ['N° Disparo', 'Disparo - Date Time']]
df_sismicidad_con_disparo = df_sismicidad_con_disparo[columns_order]
df_sismicidad_con_disparo.rename(columns={'Date Time': 'Sismicidad - Date Time'}, inplace=True)

# Eliminar las filas que no tienen N° Disparo asignado
df_sismicidad_con_disparo = df_sismicidad_con_disparo.dropna(subset=['N° Disparo']).reset_index(drop=True)


# Calcular Mo acumulado por disparo
df_sismicidad_con_disparo["Mo_cumulative"] = (
    df_sismicidad_con_disparo.groupby("N° Disparo")["Mo"].transform("sum")
)

# Calcular la energía acumulada por disparo
df_sismicidad_con_disparo["Energy_cumulative"] = (
    df_sismicidad_con_disparo.groupby("N° Disparo")["Energy [J]"].transform("sum")
)

# Calcular tabla unificada de datos de tronaduras y sismicidad
df_sismicidad_con_disparo['-'] = '-'
df_Tabla_Unificada = pd.merge(
    df_sismicidad_con_disparo[['N° Disparo', 'Mo_cumulative', 'Energy_cumulative', '-']],
    df_tronaduras,
    left_on="N° Disparo",
    right_on="N° Disparo",
    how="left"
)

# Eliminar la columna 'Fecha' de la tabla unificada
df_Tabla_Unificada = df_Tabla_Unificada.drop(columns=['Fecha'])

# Dejar valores unicos por N° Disparo
df_Tabla_Unificada = df_Tabla_Unificada.drop_duplicates(subset=['N° Disparo']).reset_index(drop=True)

# Ahora convertir las columnas categoricas en numericas
#categorical_columns = df_Tabla_Unificada.select_dtypes(include=['object']).columns
#print(f"Categorical columns: {categorical_columns}")


# 🔗 Obteniendo las variables que están definidas como categoricas
vd = db_helper.select_df(table="Variables_Description", schema="Raw_Data")[['Conjunto', 'Variable', 'Tipo']]
variables_categoricas = vd[
    (vd['Conjunto'] == 'Tronaduras') & 
    (vd['Tipo'] == 'categorical')][['Variable']]

# Convertir las variables categoricas a numericas
for var in variables_categoricas['Variable']:
    if var in df_Tabla_Unificada.columns:
        print(f"🔄 Convirtiendo {var} a numerico")
        df_Tabla_Unificada[var] = pd.Categorical(df_Tabla_Unificada[var]).codes
    else:
        print(f"⚠️ La columna {var} no está presente en el DataFrame")

db_helper.close_connection()

In [ ]:

#╭──────────────────────────────────────────────────────────────╮
#| Guardar los resultados en la base de datos                   |
#╰──────────────────────────────────────────────────────────────╯
# Crear las tablas en la base de datos a partir de los DataFrames:
db_helper.create_table_from_df("Tronaduras", df_tronaduras, schema="Processed_Data")
db_helper.create_table_from_df("Sismicidad", df_sismicidad_con_disparo, schema="Processed_Data")
db_helper.create_table_from_df("Tabla_Unificada", df_Tabla_Unificada, schema="Processed_Data")
db_helper.close_connection()